In [126]:
 import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils
from pathlib import Path
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from lockdowndates.core import LockdownDates

from catboost import CatBoostRegressor
from jours_feries_france import JoursFeries
from vacances_scolaires_france import SchoolHolidayDates
from datetime import timedelta, date

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [127]:
X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")

## Dense

In [136]:
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']
time_features = ['hour','month','weekday','day']

col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features),
        ('time', FunctionTransformer(utils.encode_cyclical_features), time_features)
    ],
    remainder='passthrough'
)

X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")
X.sort_values(by=['counter_id','date'],inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [137]:
cat = CatBoostRegressor(
    depth=10,
    iterations=1000,
    rsm=0.25,
    sampling_frequency="PerTree",
    subsample=0.7,
    verbose=0,
)

pipe = Pipeline([
    ('prepro',FunctionTransformer(utils.prepro)),
    ('col', col_transformer),
    ('model', cat)
])


pipe.fit(X_train, y_train)

predictions = pipe.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 0.9008118301694918


In [138]:
catboost_predictions_train = pipe.predict(X_train)
catboost_predictions_test = pipe.predict(X_test)

In [139]:
pipe2 = Pipeline([
    ('prepro',FunctionTransformer(utils.prepro)),
    ('col', col_transformer),
])
X_train_prepro = pipe2.fit_transform(X_train)
X_test_prepro = pipe2.fit_transform(X_test)

In [140]:


X_train_with_predictions = np.hstack((X_train_prepro, catboost_predictions_train.reshape(-1, 1)))
X_test_with_predictions = np.hstack((X_test_prepro, catboost_predictions_test.reshape(-1, 1)))


# Créer des séquences de longueur 24 pour le LSTM
sequence_length = 24
X_train_sequences = np.array([X_train_with_predictions[i:i+sequence_length] for i in range(len(X_train_with_predictions)-sequence_length)])
X_test_sequences = np.array([X_test_with_predictions[i:i+sequence_length] for i in range(len(X_test_with_predictions)-sequence_length)])

y_train_sequences = np.array([y_train[i+sequence_length] for i in range(len(y_train)-sequence_length)])

y_test_sequences = np.array([y_test[i+sequence_length] for i in range(len(y_test)-sequence_length)])

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, X_train_sequences.shape[2])))
model.add(Dense(1))  # Régression

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_sequences, y_train_sequences, epochs=2, batch_size=32, validation_data=(X_test_sequences, y_test_sequences))



Epoch 1/2
12420/12420 [==============================] - 49s 4ms/step - loss: 16608.3848 - val_loss: 4.2337
Epoch 2/2
12420/12420 [==============================] - 47s 4ms/step - loss: 73.2940 - val_loss: 2.8016


In [122]:
y_pred = model.predict(X_test_with_predictions)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

3106/3106 [==============================] - 1s 310us/step
Root Mean Squared Error: 0.34577425521475835


In [123]:
X_final_test_scaled = pipe.predict(X_final_test)
X_final_test_with_predictions = np.hstack((pipe2.fit_transform(X_final_test), X_final_test_scaled.reshape(-1, 1)))
submission = model.predict(X_final_test_with_predictions)
pd.DataFrame(submission, columns=["log_bike_count"]).rename_axis('Id').to_csv('submission34_deep_101223.csv')

1608/1608 [==============================] - 1s 314us/step


In [ ]:
num_features = ['temp', 'precip', 'windspeed', 'visibility']
cat_features = ['counter_id']
time_features = ['hour','month','weekday','day']



col_transformer = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(sparse=False), cat_features),
        #('time', FunctionTransformer(utils.encode_cyclical_features), time_features)
    ],
    remainder='passthrough'
)

X, y = utils.get_train_data()
X_final_test = pd.read_parquet(Path("data") / "final_test.parquet")
X_transformed = col_transformer.fit_transform(utils.prepro(X))

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

In [70]:
import pandas as pd

X, y = utils.get_train_data()
# Supposons que 'date_column' soit la colonne indiquant l'ordre temporel
# Trier les données par counter_id et date/heure
X_sorted = X.sort_values(by=['counter_id', 'date']).reset_index().drop(columns='index')
y = pd.Series(y)
# Définir la longueur de la séquence
# Choisissez une valeur qui convient à tous les counter_id, par exemple, la longueur minimale trouvée précédemment
sequence_length = 6577  # Exemple basé sur votre plus petit counter_id

# Créer des séquences pour chaque counter_id
sequences = []
targets = []

for counter_id in X_sorted['counter_id'].unique():
    counter_data = X_sorted[X_sorted['counter_id'] == counter_id]
    counter_target = y[counter_data.index]
    
    for i in range(len(counter_data) - sequence_length + 1):
        sequence = counter_data.iloc[i:i + sequence_length]
        target = counter_target.iloc[i + sequence_length - 1]
        
        sequences.append(sequence)
        targets.append(target)

# Conversion des séquences en un format adapté pour le modèle LSTM
import numpy as np

X_sequences = np.array([seq.values for seq in sequences])  # Assurez-vous que toutes les séquences ont la même taille
y_targets = np.array(targets)

# Maintenant, vous pouvez utiliser X_sequences et y_targets pour entraîner votre LSTM


KeyboardInterrupt: 

In [69]:
pd.Series(y)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().